In [11]:
import cv2
import torch
import torch.nn as nn

import matplotlib.pyplot as plt

In [2]:
model_type = "DPT_Large"     # MiDaS v3 - Large     (highest accuracy, slowest inference speed)
#model_type = "DPT_Hybrid"   # MiDaS v3 - Hybrid    (medium accuracy, medium inference speed)
#model_type = "MiDaS_small"  # MiDaS v2.1 - Small   (lowest accuracy, highest inference speed)

midas = torch.hub.load("intel-isl/MiDaS", model_type)

/home/evrachoriti/CIL/lib/python3.12/site-packages/torch/hub.py:330: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(


Downloading: "https://github.com/intel-isl/MiDaS/zipball/master" to /home/evrachoriti/.cache/torch/hub/master.zip


/home/evrachoriti/CIL/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/evrachoriti/CIL/lib/python3.12/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


Downloading: "https://github.com/isl-org/MiDaS/releases/download/v3/dpt_large_384.pt" to /home/evrachoriti/.cache/torch/hub/checkpoints/dpt_large_384.pt


100%|██████████| 1.28G/1.28G [00:15<00:00, 86.1MB/s]


In [3]:
type(midas)

midas.dpt_depth.DPTDepthModel

In [25]:
from midas.dpt_depth import DPT

In [26]:
class MiDaS_UQ(DPT):
    def __init__(self, **kwargs):
        def __init__(self, path=None, non_negative=True, **kwargs):
            features = kwargs["features"] if "features" in kwargs else 256
            head_features_1 = kwargs["head_features_1"] if "head_features_1" in kwargs else features
            head_features_2 = kwargs["head_features_2"] if "head_features_2" in kwargs else 32
            kwargs.pop("head_features_1", None)
            kwargs.pop("head_features_2", None)
    
            head = nn.Sequential(
                nn.Conv2d(head_features_1, head_features_1 // 2, kernel_size=3, stride=1, padding=1),
                Interpolate(scale_factor=2, mode="bilinear", align_corners=True),
                nn.Conv2d(head_features_1 // 2, head_features_2, kernel_size=3, stride=1, padding=1),
                nn.ReLU(True),
                nn.Conv2d(head_features_2, 2, kernel_size=1, stride=1, padding=0),
                # nn.ReLU(True) if non_negative else nn.Identity(),
                # nn.Identity(),
            )
    
            super().__init__(head, **kwargs)
    
            if path is not None:
               self.load(path)

        def forward(self, x):
            return super().forward(x).squeeze(dim=1)

In [27]:
class DepthUncertaintyLoss(nn.Module):
    def __init__(self, eps=1e-6):
        super().__init__()
        self.gaussian_nll = nn.GaussianNLLLoss(eps=eps)

    def forward(self, depth_pred, logvar_depth, depth_gr):
        var = torch.exp(logvar_pred).clamp(min=1e-6)
        return self.gaussian_nll(depth_pred, depth_gt, var)

In [29]:
midas_uq = MiDaS_UQ()